# PyCity Schools Analysis

* Overall, the majority (65%) of students in PyCity are proficient in both math and reading.

* Per pupil spending doesn't appear to be a major driver of academic returns, as the overall passing rate for the lowest-spending schools is 36 percentage points (90% vs 54%) higher than the most expensive school.

* The analysis lacks student socio-demographic information, which would be needed to analyze the link between per pupil spending and results. It is possible that higher-spending schools have a higher proportion of special-needs students who require more expensive learning supports while still tending towards lower scores. 

* Smaller schools and charter school both outperformed. Given the level of anlaysis performed, it is impossible to say which is the larger driving factor. The analysis of best-performing schools versus lowest-performing schools shows that the best-performing schools are charter schools whose student populations are well below the lowest performing schools. I recommend the district undertake further research to determine if the driving factor is pedagogical differences between school types, or if district schools could have similar results be decreasing the size of its high schools.

* The lack of student demographic information is a substantial limitation of the analysis. Possible charter school admission policies and/or disciplinary policy differences could result in substantial differences in the student populations relatice to the district which must serve all students.  

In [149]:
#dependencies
import pandas as pd

In [150]:
#set filepath as variable
school_file = 'Resources/schools_complete.csv'
student_file = 'Resources/students_complete.csv'

#read csv and save in dataframe
school_df = pd.read_csv(school_file)
student_df = pd.read_csv(student_file)


In [151]:
#combine into one dataset
complete_df = pd.merge(student_df, school_df, on='school_name', how='left')
complete_df.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


## District Summary

In [152]:
#Calculate total number of unique schools
school_count = (complete_df['school_name'].nunique())
school_count

15

In [153]:
#calculate total number of students
student_count = complete_df['student_name'].count()
student_count

39170

In [154]:
#calculate total budget
total_budget = school_df['budget'].sum()
total_budget

24649428

In [155]:
#calculate average math score
ave_math = complete_df['math_score'].mean()
ave_math

78.98537145774827

In [156]:
#calcualte the average reading score
ave_read = complete_df['reading_score'].mean()
ave_read

81.87784018381414

In [157]:
#calculate percentage of students who passed math
passed_math = complete_df.loc[complete_df['math_score'] >= 70].count()['student_name']
passed_math_percent = passed_math/student_count * 100
passed_math_percent

74.9808526933878

In [158]:
#calculate percentage of student who passed reading
passed_read = complete_df.loc[complete_df['reading_score'] >= 70].count()['student_name']
passed_read_percent = passed_read/student_count * 100
passed_read_percent

85.80546336482001

In [159]:
#calculate percentage that passed math and reading
passed_both = complete_df.loc[(complete_df['math_score'] >= 70) & (complete_df['reading_score'] >= 70)].count()['student_name']
passed_both_percent = passed_both/student_count * 100
passed_both_percent

65.17232575950983

In [160]:
#create high-level snapshot of district("${:,.2f}".format)
district_summary = pd.DataFrame({'Total Schools': school_count,
                                'Total Students': student_count,
                                'Total Budget': total_budget,
                                 'Average Math Score': ave_math,
                                 'Average Reading Score': ave_read,
                                 '% Passing Math': passed_math_percent,
                                 '% Passing Reading': passed_read_percent,
                                 '% Overall Passing': passed_both_percent}, index = [0])

#Formatting
district_summary['Total Students'] = district_summary['Total Students'].map("{:,}".format)
district_summary['Total Budget'] = district_summary['Total Budget'].map("${:,.2f}".format)

#Display DataFrame
district_summary

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",78.985371,81.87784,74.980853,85.805463,65.172326


## School Summary   

In [161]:
# Use the code provided to select the school type
school_types = school_df.set_index(["school_name"])['type']
school_types.head()

school_name
Huang High School        District
Figueroa High School     District
Shelton High School       Charter
Hernandez High School    District
Griffin High School       Charter
Name: type, dtype: object

In [162]:
#calculate total students per school
students_ps = complete_df.groupby('school_name')['student_name'].count()

In [163]:
#calculate total school budget
school_budget = complete_df.groupby('school_name')['budget'].mean()

In [164]:
#calculate per student budget
budget_ps = school_budget/students_ps

In [165]:
#calculate average math score per school
ave_math_ps = complete_df.groupby('school_name')['math_score'].mean()

In [166]:
#calculate average reading score per school
ave_reading_ps = complete_df.groupby('school_name')['reading_score'].mean()

In [167]:
#calculate % of student passing math at each school
passed_math_ps = complete_df[complete_df['math_score']>=70]
ps_passed_math_percent = passed_math_ps.groupby(['school_name']).count()['student_name'] / students_ps *100

In [168]:
#calculate % of student passing reading at each school
passed_read_ps = complete_df[complete_df['reading_score']>=70]
ps_passed_read_percent = passed_read_ps.groupby(['school_name']).count()['student_name'] / students_ps *100

In [169]:
#calculate the overall passing rate (students who passed both math and reading) per school
overall_pass_ps = complete_df[(complete_df['math_score']>=70) & (complete_df['reading_score']>=70)]
overall_pass_ps_percent = overall_pass_ps.groupby(['school_name']).count()['student_name'] / students_ps *100

In [170]:
# Create a DataFrame called `per_school_summary` with columns for the calculations above.
per_school_summary = pd.DataFrame({'School Type': school_types, 
                                'Total Students': students_ps,
                                'Total School Budget': school_budget,
                                'Per Student Budget': budget_ps,
                                'Average Math Score': ave_math_ps,
                                'Average Reading Score': ave_reading_ps,
                                '% Passing Math': ps_passed_math_percent,
                                '% Passing Reading': ps_passed_read_percent,
                                '% Overall Passing': overall_pass_ps_percent})

#copy database prior to formatting for anlaysis binning by per student spending
per_school_summary2 = per_school_summary.copy()

#Formatting
per_school_summary["Total School Budget"] = per_school_summary["Total School Budget"].map("${:,.2f}".format)
per_school_summary["Per Student Budget"] = per_school_summary["Per Student Budget"].map("${:,.2f}".format)

#Display DataFrame
per_school_summary

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,54.642283
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,54.289887
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,92.505855,96.252927,89.227166
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,90.540541


## Highest-Performing Schools (by % Overall Passing)

In [171]:
# Sort the schools by `% Overall Passing` in descending order and display the top 5 rows.
highest_performers = per_school_summary.sort_values('% Overall Passing', ascending=False)
highest_performers.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.418349,83.848930,93.272171,97.308869,90.948012
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,90.582567
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,90.540541


## Bottom Performing Schools (By % Overall Passing)

In [172]:
# Sort the schools by `% Overall Passing` in ascending order and display the top 5 rows.
lowest_performers = per_school_summary.sort_values('% Overall Passing')
lowest_performers.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,52.988247
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172


## Math Score by Grade

In [173]:
# Use the code provided to separate the data by grade
ninth_graders = complete_df[(complete_df["grade"] == "9th")]
tenth_graders = complete_df[(complete_df["grade"] == "10th")]
eleventh_graders = complete_df[(complete_df["grade"] == "11th")]
twelfth_graders = complete_df[(complete_df["grade"] == "12th")]

# Group by "school_name" and take the mean of each.
ninth_graders_scores = ninth_graders.groupby(['school_name']).mean() 
tenth_graders_scores = tenth_graders.groupby(['school_name']).mean()
eleventh_graders_scores = eleventh_graders.groupby(['school_name']).mean()
twelfth_graders_scores = twelfth_graders.groupby(['school_name']).mean()

# Use the code to select only the `math_score`.
ninth_grade_math_scores = ninth_graders_scores["math_score"]
tenth_grader_math_scores = tenth_graders_scores["math_score"]
eleventh_grader_math_scores = eleventh_graders_scores["math_score"]
twelfth_grader_math_scores = twelfth_graders_scores["math_score"]

# Combine each of the scores above into single DataFrame called `reading_scores_by_grade`
math_scores_by_grade = pd.DataFrame({'9th': ninth_grade_math_scores,
                                        '10th': tenth_grader_math_scores,
                                        '11th': eleventh_grader_math_scores,
                                        '12th': twelfth_grader_math_scores})

# Minor data wrangling
math_scores_by_grade.index.name = None

# Display the DataFrame
math_scores_by_grade

,9th,10th,11th,12th
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248
Pena High School,83.625455,83.372000,84.328125,84.121547


## Reading Score by Grade

In [174]:
# Use the code provided to separate the data by grade
ninth_graders = complete_df[(complete_df["grade"] == "9th")]
tenth_graders = complete_df[(complete_df["grade"] == "10th")]
eleventh_graders = complete_df[(complete_df["grade"] == "11th")]
twelfth_graders = complete_df[(complete_df["grade"] == "12th")]

# Group by "school_name" and take the mean of each.
ninth_graders_scores = ninth_graders.groupby(['school_name']).mean() 
tenth_graders_scores = tenth_graders.groupby(['school_name']).mean()
eleventh_graders_scores = eleventh_graders.groupby(['school_name']).mean()
twelfth_graders_scores = twelfth_graders.groupby(['school_name']).mean()

# Use the code to select only the `reading_score`.
ninth_grade_reading_scores = ninth_graders_scores["reading_score"]
tenth_grader_reading_scores = tenth_graders_scores["reading_score"]
eleventh_grader_reading_scores = eleventh_graders_scores["reading_score"]
twelfth_grader_reading_scores = twelfth_graders_scores["reading_score"]

# Combine each of the scores above into single DataFrame called `reading_scores_by_grade`
reading_scores_by_grade = pd.DataFrame({'9th': ninth_grade_reading_scores,
                                        '10th': tenth_grader_reading_scores,
                                        '11th': eleventh_grader_reading_scores,
                                        '12th': twelfth_grader_reading_scores})

# Minor data wrangling
reading_scores_by_grade.index.name = None

# Display the DataFrame
reading_scores_by_grade

,9th,10th,11th,12th
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564
Pena High School,83.807273,83.612000,84.335938,84.591160


## Scores by School Spending

In [175]:
# Establish the bins 
spending_bins = [0, 585, 630, 645, 680]
labels = ["<$585", "$585-630", "$630-645", "$645-680"]

In [176]:
# Create a copy of the school summary since it has the "Per Student Budget". 
school_spending_df = per_school_summary2.copy()

In [177]:
#Use `pd.cut` to categorize spending based on the bins.
school_spending_df["Spending Ranges (Per Student)"] = pd.cut(school_spending_df['Per Student Budget'], spending_bins, labels = labels, include_lowest=True)


In [178]:
#  Calculate averages for the desired columns. 
spending_math_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"]).mean()["Average Math Score"]
spending_reading_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"]).mean()["Average Reading Score"]
spending_passing_math = school_spending_df.groupby(["Spending Ranges (Per Student)"]).mean()["% Passing Math"]
spending_passing_reading = school_spending_df.groupby(["Spending Ranges (Per Student)"]).mean()["% Passing Reading"]
overall_passing_spending = school_spending_df.groupby(["Spending Ranges (Per Student)"]).mean()["% Overall Passing"]

In [179]:
# Assemble into DataFrame
spending_summary_df = pd.DataFrame({'Average Math Score': spending_math_scores,
                                'Average Reading Score':spending_reading_scores,
                                '% Passing Math': spending_passing_math,
                                "% Passing Reading": spending_passing_reading,
                                '% Overall Passing': overall_passing_spending})

# Display results
spending_summary_df

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Spending Ranges (Per Student),,,,,
<$585,83.455399,83.933814,93.460096,96.610877,90.369459
$585-630,81.899826,83.155286,87.133538,92.718205,81.418596
$630-645,78.518855,81.624473,73.484209,84.391793,62.857656
$645-680,76.997210,81.027843,66.164813,81.133951,53.526855


## Scores by School Size

In [180]:
# Establish the bins.
size_bins = [0, 1000, 2000, 5000]
size_labels = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [181]:
# Categorize the spending based on the bins
# Use `pd.cut` on the "Total Students" column of the `per_school_summary` DataFrame.

per_school_summary2["School Size"] = pd.cut(per_school_summary2['Total Students'], size_bins, labels = size_labels, include_lowest=True)


In [182]:
# Calculate averages for the desired columns. 
size_math_scores = per_school_summary2.groupby(["School Size"]).mean()["Average Math Score"]
size_reading_scores = per_school_summary2.groupby(["School Size"]).mean()["Average Reading Score"]
size_passing_math = per_school_summary2.groupby(["School Size"]).mean()["% Passing Math"]
size_passing_reading = per_school_summary2.groupby(["School Size"]).mean()["% Passing Reading"]
size_overall_passing = per_school_summary2.groupby(["School Size"]).mean()["% Overall Passing"]

In [183]:
# Create a DataFrame called `size_summary` that breaks down school performance based on school size (small, medium, or large).
size_summary = pd.DataFrame({'Average Math Score': size_math_scores,
                            'Average Reading Score': size_reading_scores,
                            '% Passing Math': size_passing_math,
                            '% Passing Reading': size_passing_reading,
                            '% Overall Passing': size_overall_passing})

# Display results
size_summary

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Size,,,,,
Small (<1000),83.821598,83.929843,93.550225,96.099437,89.883853
Medium (1000-2000),83.374684,83.864438,93.599695,96.790680,90.621535
Large (2000-5000),77.746417,81.344493,69.963361,82.766634,58.286003


## Scores by School Type

In [184]:
# Group the per_school_summary DataFrame by "School Type" and average the results.
type_math_scores = per_school_summary2.groupby('School Type')[['Average Math Score']].mean()
type_reading_scores = per_school_summary2.groupby('School Type')[['Average Reading Score']].mean()
type_passing_math = per_school_summary2.groupby('School Type')[['% Passing Math']].mean() 
type_passing_reading = per_school_summary2.groupby('School Type')[['% Passing Reading']].mean()
type_overall_passing = per_school_summary2.groupby('School Type')[['% Overall Passing']].mean()



In [185]:
# Use the code provided to select new column data
average_math_score_by_type = type_math_scores["Average Math Score"]
average_reading_score_by_type = type_reading_scores["Average Reading Score"]
average_percent_passing_math_by_type = type_passing_math["% Passing Math"]
average_percent_passing_reading_by_type = type_passing_reading["% Passing Reading"]
average_percent_overall_passing_by_type = type_overall_passing["% Overall Passing"]



In [186]:
# Assemble the new data by type into a DataFrame called `type_summary`
type_summary = pd.DataFrame([average_math_score_by_type, 
                            average_reading_score_by_type, 
                            average_percent_passing_math_by_type,
                            average_percent_passing_reading_by_type,
                            average_percent_overall_passing_by_type]).transpose()

# Display results
type_summary

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,90.432244
District,76.956733,80.966636,66.548453,80.799062,53.672208
